### Importing Required Libraries

In [61]:
import pandas as pd
import datetime as dt 

### Read the 'Seattle Paid Parking Occcupany' Last 48 hrs Dataset into a Pandas Dataframe

In [62]:
df=pd.read_csv("C:\Step 3 - DataCollection\PaidParkingOccupancy.csv")

C:\Users\yogit\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Getting the first 3 records of the Dataframe

In [63]:
df.head(3)

,OccupancyDateTime,PaidOccupancy,BlockfaceName,SideOfStreet,SourceElementKey,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,PaidParkingRate,ParkingCategory,Location
0,11/24/2020 02:33:00 PM,2,S WELLER ST BETWEEN MAYNARD AVE S AND MAYNARD ...,N,"66,617",120,4,Chinatown/ID,Core,NaN,Paid Parking,POINT (-122.32469977 47.59758631)
1,11/24/2020 03:20:00 PM,7,WESTERN AVE BETWEEN VIRGINIA ST AND LENORA ST,NE,"58,806",120,6,Belltown,South,NaN,Paid Parking,POINT (-122.34403853 47.61082404)
2,11/24/2020 03:17:00 PM,1,E PINE ST BETWEEN CRAWFORD PL AND SUMMIT AVE,S,"37,174",120,3,Pike-Pine,NaN,NaN,Paid Parking,POINT (-122.32587891 47.61514809)


### Data Types of the columns in the Dataframe

In [64]:
df.dtypes

OccupancyDateTime            object
PaidOccupancy                 int64
BlockfaceName                object
SideOfStreet                 object
SourceElementKey             object
ParkingTimeLimitCategory     object
ParkingSpaceCount             int64
PaidParkingArea              object
PaidParkingSubArea           object
PaidParkingRate             float64
ParkingCategory              object
Location                     object
dtype: object

### Checking missing values

In [65]:
df.isnull().any()

OccupancyDateTime           False
PaidOccupancy               False
BlockfaceName               False
SideOfStreet                False
SourceElementKey            False
ParkingTimeLimitCategory     True
ParkingSpaceCount           False
PaidParkingArea             False
PaidParkingSubArea           True
PaidParkingRate              True
ParkingCategory             False
Location                    False
dtype: bool

In [66]:
print("'PaidParkingSubArea' have blank values as every Parking Area doesn't have a sub-area")
print("'PaidParkingRate' have null values and can be dropped as this information will be captured via Blockface dataset")

'PaidParkingSubArea' have blank values as every Parking Area doesn't have a sub-area
'PaidParkingRate' have null values and can be dropped as this information will be captured via Blockface dataset


### There are some Blank and out of range values like 4320 in the 'ParkingTimeLimitCategory' column
### These values match with the Blockface column data as shown in the following steps

In [67]:
uniqueValues = df['ParkingTimeLimitCategory'].unique()
print('Unique elements in column "ParkingTimeLimitCategory" ')
print(uniqueValues)

Unique elements in column "ParkingTimeLimitCategory" 
[120.0 600.0 240.0 30.0 nan '600' '240' '120' '30' '4,320']


## Rename the column value

In [68]:
df.rename(columns = {'SourceElementKey':'Station_ID','PaidOccupancy':'Occupied_Spots','ParkingSpaceCount':'Available_Spots'}, inplace = True) 

In [69]:
df.dtypes

OccupancyDateTime            object
Occupied_Spots                int64
BlockfaceName                object
SideOfStreet                 object
Station_ID                   object
ParkingTimeLimitCategory     object
Available_Spots               int64
PaidParkingArea              object
PaidParkingSubArea           object
PaidParkingRate             float64
ParkingCategory              object
Location                     object
dtype: object

## Finally schema needs to be changed as follows

|Column|DataType| Desciption|
|-----|-------|-------------|
|OccupancyDateTime|DATETIME|The date and time (minute) of the transaction as recorded|
|Occupied_Spots|INT|Number of transactions that have paid at this date/time on this blockface|
|BlockfaceName|STRING|Street segment, name of street with the “from street” and “to street|
|SideOfStreet|STRING|Options are: E, S, N, W, NE, SW, SE, NW.|
|Station_ID|INT|Unique identifier for the city street segment where the pay station is located;|
|ParkingTimeLimitCategory|INT|In minutes. Options are 120 (2-hour parking), 240 (4-hour parking), 30, or 600 (10-hour parking)|
|Available_Spots|INT|Number of paid spaces on the blockface at the given date and time.|
|PaidParkingArea|STRING|The primary name of a paid parking neighborhood. Example is Commercial Core.|
|PaidParkingSubArea|STRING|A subset of a paid parking area—not all paid parking areas have subareas.|
|PaidParkingRate|FLOAT|Parking rate charged at date and time.|
|ParkingCategory|STRING|An overall description of the type of parking allowed on a blockface.|
|Location|STRING|Calculated based on the known location of a pay station along the same blockface.|

### Data Cleaning and Adding Additional Columns in Seattle Parking Occupancy Data
<ol>
<li>Create the OccupancyTimeData Dimension Table using OccupancyDateTime and derive the additional columns such as 'month','day of the week','hour' using Pyspark to get the monthly,daily and hourly trends</li>
<li>Remove the comma from the source element id</li>
<li>Drop unwanted columns such as 'BlockfaceName','SideOfStreet','ParkingTimeLimitCategory', etc</li>
</ol>

### Final Occupancy Fact Table should look like this

|Column|DataType| Desciption|
|-----|-------|-------------|
|Station_ID|INT|Unique identifier for the city street segment where the pay station is located|
|Occupied_Spots|INT|Number of transactions that have paid at this date/time on this blockface|
|Available_Spots|INT|Number of paid spaces on the blockface at the given date and time.|
|OccupancyDateTime|DATETIME|The date and time (minute) of the transaction as recorded|

### OccupancyDateTime Dimension Table

|Column|DataType| Desciption|
|-----|-------|-------------|
|OccupancyDateTime|DATETIME|The date and time (minute) of the transaction as recorded|
|month|INT|Month when the spot was occupied|
|day_of_the_week|STRING|Day of the week when the spot was occupied. Valid values are Mon, Tue,Wed,Thurs, Fri,Sat & Sun|
|hour|INT|Hour when the spot was occupied|

### Blockface Data

### Displays blockfaces for all segments of the street network. Identifies the elements of the block, such as peak hour restrictions, length of the block, parking categories, and restricted parking zones.  


In [15]:
blockface=pd.read_csv("C:\Step 3 - DataCollection\Blockface.csv")

C:\Users\yogit\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (34,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [26]:
blockface.head(3)

,ELMNTKEY,UNITDESC,SIDE,BLOCK_NBR,PARKING_CATEGORY,WKD_RATE1,WKD_START1,WKD_END1,WKD_RATE2,WKD_START2,...,SAT_START1,SAT_END1,SAT_RATE2,SAT_START2,SAT_END2,SAT_RATE3,SAT_START3,SAT_END3,PARKING_TIME_LIMIT,SUBAREA
0,122148,DUWAMISH RIVER TRL BETWEEN WEST MARGINAL WAY S...,W,4200,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,137002,SR99 TUNNEL NB BETWEEN ALASKAN WY VI NB AND SR...,W,0,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,136762,6TH AV N SR99 ON RP BETWEEN 6TH AVE N AND SR99...,W,0,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [17]:
blockface.dtypes

OBJECTID                 int64
ELMNTKEY                 int64
SEGKEY                   int64
UNITID                   int64
UNITID2                  int64
UNITDESC                object
SIDE                    object
BLOCK_ID                object
BLOCK_NBR                int64
CSM                     object
PARKING_CATEGORY        object
LOAD                     int64
ZONE                     int64
TOTAL_ZONES              int64
WKD_RATE1              float64
WKD_START1               int64
WKD_END1                 int64
WKD_RATE2              float64
WKD_START2               int64
WKD_END2                 int64
WKD_RATE3              float64
WKD_START3             float64
WKD_END3               float64
SAT_RATE1              float64
SAT_START1             float64
SAT_END1               float64
SAT_RATE2              float64
SAT_START2             float64
SAT_END2               float64
SAT_RATE3              float64
SAT_START3             float64
SAT_END3               float64
RPZ_ZONE

### Drop unwanted columns

In [18]:
blockface = blockface.drop(["OBJECTID","SEGKEY",
                               "UNITID",
                                "UNITID2",
                                "BLOCK_ID",
                                "CSM",
                                "LOAD",
                                "ZONE",
                                "TOTAL_ZONES",
                                "RPZ_ZONE",
                                "RPZ_AREA",
                                "PAIDAREA",
                                "START_TIME_WKD",
                                "END_TIME_WKD",
                                "START_TIME_SAT",
                                "END_TIME_SAT",
                                "PRIMARYDISTRICTCD",
                                "SECONDARYDISTRICTCD",
                                "OVERRIDEYN",
                                "OVERRIDECOMMENT",
                                "SHAPE_Length"],axis=1)

In [19]:
blockface.head(3)

,ELMNTKEY,UNITDESC,SIDE,BLOCK_NBR,PARKING_CATEGORY,WKD_RATE1,WKD_START1,WKD_END1,WKD_RATE2,WKD_START2,...,SAT_START1,SAT_END1,SAT_RATE2,SAT_START2,SAT_END2,SAT_RATE3,SAT_START3,SAT_END3,PARKING_TIME_LIMIT,SUBAREA
0,122148,DUWAMISH RIVER TRL BETWEEN WEST MARGINAL WAY S...,W,4200,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,137002,SR99 TUNNEL NB BETWEEN ALASKAN WY VI NB AND SR...,W,0,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,136762,6TH AV N SR99 ON RP BETWEEN 6TH AVE N AND SR99...,W,0,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [30]:
blockface.dtypes

ELMNTKEY                int64
UNITDESC               object
SIDE                   object
BLOCK_NBR               int64
PARKING_CATEGORY       object
WKD_RATE1             float64
WKD_START1              int64
WKD_END1                int64
WKD_RATE2             float64
WKD_START2              int64
WKD_END2                int64
WKD_RATE3             float64
WKD_START3            float64
WKD_END3              float64
SAT_RATE1             float64
SAT_START1            float64
SAT_END1              float64
SAT_RATE2             float64
SAT_START2            float64
SAT_END2              float64
SAT_RATE3             float64
SAT_START3            float64
SAT_END3              float64
PARKING_TIME_LIMIT    float64
SUBAREA                object
dtype: object

### There are blank values for certain columns like WKD_RATE3/WKDSTART3/WKDEND3/SAT_RATE1 as there could be no parking permitted during that time period in those areas

In [22]:
blockface.isnull().any()

ELMNTKEY              False
UNITDESC              False
SIDE                  False
BLOCK_NBR             False
PARKING_CATEGORY      False
WKD_RATE1             False
WKD_START1            False
WKD_END1              False
WKD_RATE2             False
WKD_START2            False
WKD_END2              False
WKD_RATE3              True
WKD_START3             True
WKD_END3               True
SAT_RATE1              True
SAT_START1             True
SAT_END1               True
SAT_RATE2              True
SAT_START2             True
SAT_END2               True
SAT_RATE3              True
SAT_START3             True
SAT_END3               True
PARKING_TIME_LIMIT     True
SUBAREA                True
dtype: bool

In [76]:
blockface.rename(columns={'ELMNTKEY':'Station_ID','UNITDESC':'UnitDesc','SIDE':'Side','BLOCK_NBR':'Block_Nbr','PARKING_CATEGORY':'Parking_Category',
                  'WKD_RATE1':'Wkd_Rate1','WKD_START1':'Wkd_Start1','WKD_END1':'Wkd_End1','WKD_RATE2':'Wkd_Rate2',
                   'WKD_START2':'Wkd_Start2','WKD_END2':'Wkd_End2','WKD_RATE3':'Wkd_Rate3','WKD_START3':'Wkd_Start3',
                   'WKD_END3':'Wkd_End3','SAT_RATE1':'Sat_Rate1','SAT_START1':'Sat_Start1','SAT_END1':'Sat_End1',
                    'SAT_RATE2':'Sat_Rate2','SAT_START2':'Sat_Start2','SAT_END2':'Sat_End2','SAT_RATE3':'Sat_Rate3',
                     'SAT_START3':'Sat_Start3','SAT_END3':'Sat_End3','PARKING_TIME_LIMIT':'Parking_Time_Limit',
                     'SUBAREA':'SubArea'},inplace=True)

In [27]:
blockface.head(3)

,ELMNTKEY,UNITDESC,SIDE,BLOCK_NBR,PARKING_CATEGORY,WKD_RATE1,WKD_START1,WKD_END1,WKD_RATE2,WKD_START2,...,SAT_START1,SAT_END1,SAT_RATE2,SAT_START2,SAT_END2,SAT_RATE3,SAT_START3,SAT_END3,PARKING_TIME_LIMIT,SUBAREA
0,122148,DUWAMISH RIVER TRL BETWEEN WEST MARGINAL WAY S...,W,4200,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,137002,SR99 TUNNEL NB BETWEEN ALASKAN WY VI NB AND SR...,W,0,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,136762,6TH AV N SR99 ON RP BETWEEN 6TH AVE N AND SR99...,W,0,None,0.0,0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [28]:
blockface.dtypes

ELMNTKEY                int64
UNITDESC               object
SIDE                   object
BLOCK_NBR               int64
PARKING_CATEGORY       object
WKD_RATE1             float64
WKD_START1              int64
WKD_END1                int64
WKD_RATE2             float64
WKD_START2              int64
WKD_END2                int64
WKD_RATE3             float64
WKD_START3            float64
WKD_END3              float64
SAT_RATE1             float64
SAT_START1            float64
SAT_END1              float64
SAT_RATE2             float64
SAT_START2            float64
SAT_END2              float64
SAT_RATE3             float64
SAT_START3            float64
SAT_END3              float64
PARKING_TIME_LIMIT    float64
SUBAREA                object
dtype: object

### Schema to be changed as follows


|Column|DataType|Description|
|-----|-------|-------------|
|Station_id| INT| Unique identifier for the city street segment (blockface)|
|UnitDesc|STRING| Name of the Street 'from_street' to 'to_street'|
|Side| STRING| Options are: E, S, N, W, NE, SW, SE, NW.|
|Block_Nbr|INT| Block Number|
|Parking_Category| STRING| An overall description of the type of parking allowed on a blockface.|
|Wkd_Rate1| DOUBLE| Weekday parking rate – 1st interval|
|Wkd_Start1| HH:MM:00|Weekday 1st interval start|
|Wkd_End1| HH:MM:00| Weekday 1st interval end |
|Wkd_Rate2| DOUBLE| Weekday parking rate -2nd interval|
|Wkd_Start2| HH:MM:00| Weekday 2nd interval start|
|Wkd_End2| HH:MM:00| Weekday 2nd interval end|
|Wkd_Rate3| DOUBLE| Weekday parking rate - 3rd interval|
|Wkd_Start3| HH:MM:00| Weekday 3rd interval start |
|Wkd_End3| HH:MM:00|Weekday 3rd interval end |
|Sat_Rate1| DOUBLE|Sat parking rate -1st Interval|
|Sat_Start1| HH:MM:00| Sat 1st interval start |
|Sat_End1| HH:MM:00| Sat 1st interval end |
|Sat_Rate2| DOUBLE| Sat parking rate - 2nd interval|
|Sat_Start2|HH:MM:00 | Sat 2nd interval start|
|Sat_End2|HH:MM:00 | Sat 2nd interval end|
|Sat_Rate3| DOUBLE| Sat parking rate - 3rd interval|
|Sat_Start3|HH:MM:00|Sat 3rd interval start| 
|Sat_End3|HH:MM:00| Sat 3rd interval end |
|Parking_Time_Limit| INT| In minutes. Options are 120 (2-hour parking), 240 (4-hour parking), 30, or 600 (10-hour parking|
|SubArea| STRING| Sub area of Parking Area|

### Data Storage

### Below are processed files

<ol>
<li>Paid Parking Occupancy 2020-18GB</li>
<li>Blockface-8MB</li>
</ol>

### There are three ways to store the processed data

<ol>
<li>CSV</li>
<li>Apache Parquet</li>
<li>Apache Avro</li>
</ol>

### The Paid Parking occupancy data generated for 2020 is lesser than the previous year i.e. 18 GB (137M rows) is likely due to travel restrictions from the ongoing pandemic. The previous year 2019 data was on the higher side i.e. 45GB(286M)

### CSV

> CSV works well for a small data set of 70,000 lines. However, what if you need to increase to 2 million records? What if each record has nested properties? While CSV files are simple and human-readable, they unfortunately do not scale well. As the file size grows, load times become impractical, and reads cannot be optimized. Their primitive nature allows them to be a great option for smaller data sets, but very inconvenient for managing larger sets of data. This is where both Parquet and Avro come in.

### Apache Parquet

>"Apache Parquet is a free and open-source column-oriented data storage format of the Apache Hadoop ecosystem. It is similar to the other columnar-storage file formats available in Hadoop namely RCFile and ORC. It provides efficient data compression and encoding schemes with enhanced performance to handle complex data in bulk."

<ul>
<li>Since it’s a column based format, it’s better to use when you only need to access specific fields</li>
<li>Each data file contains the values for a set of rows</li>
<li>Can’t be written from streaming data since it needs to wait for blocks to get finished. However, this will work using micro-batch (eg Apache Spark).
</li>
<li>Suitable for data exploration — read intensive, complex or analytical querying, low latency data</li>
</ul>

### Apache Avro

> Apache Avro is a remote procedure call and data serialization framework developed within Apache’s Hadoop project. It uses JSON for defining data types and protocols, and serializes data in a compact binary format.

<ul>
<li>Since it’s a row based format, it’s better to use when all fields needs to be accessed</li>
<li>Files support block compression and are splittable</li>
<li>Can be written from streaming data (eg Apache Flink)</li>
<li>Suitable for write intensive operation</li>
</ul>

### Data Storage Format Conclusion

### For Seattle Parking Occupancy project Apache Parquet is better suited for data storage as we deal with immutable data and analytical queries, for which columnar storage is optimal.

### Paid Parking Transaction Data

### TBD: Real Time processing

In [34]:
paid_parking=pd.read_csv("C:\Step 3 - DataCollection\Paid_Parking_Transaction_Data.csv")

In [35]:
paid_parking.head(5)

,Transaction ID,Meter Code,Transaction DateTime,Payment Mean,Amount Paid,Duration In Minutes,Blockface Name,Side Of Street,Element key,Parking Space Number,Latitude,Longitude
0,910452505,4501002,11/25/2020 04:52:00 AM,CREDIT CARD,1.0,120,MADISON ST BETWEEN 3RD AVE AND 4TH AVE,SE,34214,NaN,47.605808,-122.333418
1,910452607,10255004,11/25/2020 04:53:00 AM,CREDIT CARD,2.0,240,4TH AVE BETWEEN VINE ST AND CEDAR ST,NE,6198,NaN,47.617572,-122.347464
2,910452836,5510002,11/25/2020 04:55:00 AM,CREDIT CARD,1.0,120,1ST AVE BETWEEN SENECA ST AND UNIVERSITY ST,SW,1013,NaN,47.606424,-122.337982
3,910452996,2020002,11/25/2020 04:56:00 AM,CREDIT CARD,1.0,120,WESTERN AVE BETWEEN SPRING ST AND SENECA ST,SW,58801,NaN,47.605281,-122.338434
4,910452997,4081002,11/25/2020 04:56:00 AM,CASH,1.0,120,COLUMBIA ST BETWEEN 4TH AVE AND 5TH AVE,SE,9770,NaN,47.604845,-122.331004


In [36]:
paid_parking.dtypes

Transaction ID            int64
Meter Code                int64
Transaction DateTime     object
Payment Mean             object
Amount Paid             float64
Duration In Minutes       int64
Blockface Name           object
Side Of Street           object
Element key               int64
Parking Space Number    float64
Latitude                float64
Longitude               float64
dtype: object

## Schema needs to be changed as follows

                                    
|Column|DataType| Description|
|-----|-------|--------------|
|Transaction ID|INT|The unique transaction identifier|
|Meter Code|INT|Unique identifier of the pay station|
|Transaction DateTime|YYYY-MM-DD HH:MM:SS|The date and time of the transaction as recorded|
|Payment Mean|STRING|Type of payment (e.g. COINS, CREDIT CARD, PHONE)|
|Amount Paid|FLOAT|The amount of the transactions in dollar|
|Duration In Minutes|INT|The calculated transaction duration (minutes)|
|Blockface Name |STRING|Street segment, name of street with the “from street” and “to street|
|Side Of Street|STRING|Options are: E, S, N, W, NE, SW, SE, NW|
|Station_ID|INT|Unique identifier for the city street segment where the pay station is located|
|Parking Space Number |FLOAT|Parking Space Number|
|Latitude|FLOAT|Latitude|
|Longitude|FLOAT|Longitude|

In [37]:
paid_parking.isnull().any()

Transaction ID          False
Meter Code              False
Transaction DateTime    False
Payment Mean            False
Amount Paid             False
Duration In Minutes     False
Blockface Name          False
Side Of Street          False
Element key             False
Parking Space Number     True
Latitude                False
Longitude               False
dtype: bool

In [39]:
paid_parking.rename(columns={'Transaction ID':'Transaction_ID','Meter Code':'Meter_Code','Transaction DateTime':'Transaction_Date_Time',
                             'Payment Mean':'Payment_Mean','Amount Paid':'Amount_Paid','Duration In Minutes':'Duration_In_Minutes',
                             'Blockface Name':'Blockface_Name','Side of Street':'Side_of_Street','Element key':'Station_ID',
                              'Parking Space Number':'Parking_Space_Number'},inplace=True)

In [55]:
paid_parking.dtypes

Transaction_ID             int64
Meter_Code                 int64
Transaction_Date_Time     object
Payment_Mean              object
Amount_Paid              float64
Duration_In_Minutes        int64
Blockface_Name            object
Side Of Street            object
Station_ID                 int64
Parking_Space_Number     float64
Latitude                 float64
Longitude                float64
dtype: object

In [45]:
amountlessthanZero = paid_parking[paid_parking['Amount_Paid']<0]

In [48]:
amountlessthanZero.head(3)

,Transaction_ID,Meter_Code,Transaction_Date_Time,Payment_Mean,Amount_Paid,Duration_In_Minutes,Blockface_Name,Side Of Street,Station_ID,Parking_Space_Number,Latitude,Longitude
50066,911347349,5088002,11/27/2020 01:07:00 PM,CASH,-1.0,120,OLIVE WAY BETWEEN 3RD AVE AND 4TH AVE,SE,12046,NaN,47.611897,-122.339017


In [49]:
durationInMins = paid_parking[paid_parking['Duration_In_Minutes']<0]

In [51]:
durationInMins.describe

<bound method NDFrame.describe of        Transaction_ID  Meter_Code   Transaction_Date_Time Payment_Mean  \
5072        910499469    18013007  11/25/2020 08:53:00 AM  CREDIT CARD   
18988       910858103    18008007  11/27/2020 06:21:00 AM  CREDIT CARD   
19014       910859919    18008007  11/27/2020 06:55:00 AM  CREDIT CARD   
19729       910867556    18005007  11/27/2020 08:16:00 AM  CREDIT CARD   
50557       911437915     5003002  11/30/2020 04:51:00 AM  CREDIT CARD   
50611       911440061    17160010  11/30/2020 05:31:00 AM  CREDIT CARD   
50952       911447439    19266002  11/30/2020 06:53:00 AM         CASH   
68816       911752646    13026002  12/01/2020 05:52:00 AM  CREDIT CARD   
69167       911764326    19127010  12/01/2020 07:08:00 AM  CREDIT CARD   

       Amount_Paid  Duration_In_Minutes  \
5072          7.00                   -7   
18988         3.50                 -159   
19014         3.50                 -125   
19729         3.50                  -44   
50557     

### Drop Rows with negative values in columns 'Amount_Paid' and 'Duration_In_Minutes'